# SCP uploading and downloading

* go to server:
     * type in terminal or command line 
     * ssh username@ip_address

* download file instruction:
    * scp username@servername:/path/filename /Users/mac/Desktop (local path)

* upload file instruction:
    * scp /path/filename username@servername:/path

* download folder instruction:
    * scp -r username@servername:/root/ (remote folder path) /Users/mac/Desktop (local path)

* upload folder instruction:
    * scp -r local_dir username@servername:remote_dir




# [YOLO](https://pjreddie.com/darknet/yolo/): Real-Time Object Detection 

## YOLO training preparation

*   generate labels and images
    * put all .jpg/.jpeg images in folder called "JPEGimages"
    * put all .txt labels in folder called "labels"
      * json to yolo format can reference this [json2yolo.py](https://github.com/yuyangh/Tools/blob/master/json2yolo.py)
    * "JPEGimages" and "labels" should be in the same directory
*   create test/train split
    * YOLO does not use validation part in training







**Code modification**
* modify Makefile
    * GPU=1 to support GPU
    * CUDNN=1 to support CUDA
    * OpenCV = 0, only when image needs OpenCV's support, then open OpenCV
*  modify model file /cfg/yolov3.cfg 
      * batch = number of samples in 1 batch
      * subdivisions = xxx means divide 1 batch to xxx subdivisions to load into GPU. 
        * Fewer subdivisions require less video memory and train slower
      * if image size is smaller than yolov3's default size, 
        * change width=416, height=416 so that model size is smaller than image size
      * max_batches = XXX, max_batches is the maximum iterations going to be trained
        * value depend on batch size: Ideally, our model expect to train up to 200-300 epoch.
        * iterations = num_epoch * (num_images/batch_size)
        * e.g. num_epoch = 200, batch_size = 64, num_images = 8862, iterations = 200 * (8862/64) ~ 30k iterations
      * for all yolo layers, change classes & filters
        * classes = number of classes you have in dataset
        * filters = (classes +5) * 3 because 5 includes object ID and bounding box
      * e.g. for yolov3, there are 3 yolo layers
*   modify cfg/xxx.data (similar to coco.data)
      * specify where your data is
      * backup= folder where you would like to backup, usually there is already a folder in /darknet/
      * eval=coco, this is the evaluation method, since we are not using that, comment this line
*  modify  data/xxx.names (similar to data/coco.names)
*  modify examples/detector.c
      * add find_replace(labelpath,".jpeg",".txt", labelpath); to support .jpeg images:
      * examples/detectors.c, on line 138, i%NUM, NUM is how frequently saved
* any update in .c files need to type **"make"** in darknet/ to update

**Start training with command: **
* make sure nobody is using that GPU before training

```
./darknet detector train cfg/flir_adas.data cfg/yolov3.cfg darknet53.conv.74 -gpus 0 &> train_output.txt
```

paramerters: 
*   cfg/yolov3.cfg is the model file
*   darknet53.conv.74 is pre-trained weights with specific choosen layers
*   -gpus 0 is choose which GPU to do training


## YOLO training process parameter meaning

* Region Avg IOU: 
  * average IOU represen the percent of intersection over union of predicted bounding box and ground truth. 
  * we expect it to approximate 1
* Class:
  * the probability to be the labeled object
  * we expect it to approximate 1
* Obj:
  * we expect it be approximate 1
* No Obj:
  * expect value should be decreasing but not 0
* Avg Recall
  * expect value approximate 0
* avg：
  * average loss, expect it be 0

## YOLO performance evaluation

*  copy yolov3.cfg to yolov3-test.cfg and modify
    * batch =1
    * subdivision =1
*  modify examples/Detector.c to do validation
    * in validate_detector_recall() function:  
    * modify threshold:  float thresh = .25;  
    * change validation path around line 496  
      * list *plist = get_paths("/mnt/large4t/pengchong_data/Data/Paul/filelist/val.txt");
    
* start evaluation
  * provided in validation.sh
  * ./darknet detector recall cfg/flir_adas.data cfg/yolov3.cfg backup/yolov3_final.weights -gpus 0 &> validation_output.txt
* measure mAP
  * git clone https://github.com/Cartucho/mAP.git
  * in extra/, 
    * for ground-truth, run  convert convert_gt_yolo.py to do ground-truth conversion
    * for detection-results, follow the guide. if darknet does not support "-dont_show" command, clone https://github.com/AlexeyAB/darknet.git, this repo supports the command
    * to change the thresh in predicton, go to src/detector.c and find "float thresh = find_float_arg(argc, argv, "-thresh", .25);" to change .25 value
    * remember to change the IMG_FORMAT = '.jpg' to your image format
    * ./darknet detector test cfg/flir_adas.data cfg/yolov3-test.cfg backup/yolov3_30000.weights -gpus 0 -dont_show -ext_output < data/flir_adas_valid.list > result.txt  
  * run main.py
     




## YOLO single image detection

* detection command
  * ./darknet detect cfg/yolov3.cfg backup/yolov3_final.weights ../FLIR_ADAS/validation/PreviewData/FLIR_08863.jpeg  
* download prediction image  
  * scp server_path:/data/projects/FireSpotter/darknet/predictions.jpg "E:\markh\Downloads"  

# Server Command

## Screening

* Why we use this? 
  * When our laptop goes to sleep or turns off, the connection to the server will be closed and session will be lost.  
  * To let the server save the connection, we need screening.
---
* Command
  * screen -S "name": attach to screen with "name"
  * screen -dr "name": switch back to command line
  * screen -ls: ls all screens
  * control A + control D: leave screen
  * exit: exit screen



## Nvidia GPU

* see GPU info: nvidia-smi
* watch GPU info: watch nvidia-smi

# Install ubuntu on PC

## preparation








* an empty hard disk with over 128 GB for the Linux system
* download ubantu operating system
    * recommend using  [ubuntu LTS version]( https://www.ubuntu.com/download/desktop )
    * choose -amd64.iso if your computer is 64 bits system
* have an empty flash driver with at least 4 GB space
    * remember to backup files
* use [rufus](https://rufus.ie/) to write the Linux operating system into the flash driver
    * in partition scheme, choose UEFI if your mother board supports UEFI, otherwise, choose MBR
        * UEFI will make the system opens faster
    * here is how to do the format
    * NOTICE: once start formating, all contents in the flash driver will be erased
* ![alt text](https://rufus.ie/pics/rufus_en.png)

## install the ubuntu system




*   go into the BIOS
    * depends on different brands, might be ESC, F2, F10, or 12
    * press one of those buttions many times after pressing the power on button
* BIOS set up
    * TURN OFF the secure boot!!!
    * this is very important, otherwise graphics driver may not install succcessful later
    * choose UEFI BIOS if your flash driver is in UEFI mode
    * put flash driver as the first one in boot order

# Vim Commands

*   search: /
*   search next: n
*   undo: u
* go the end of file: shift+G